In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import json
import time
import re


driver = webdriver.Chrome()
driver.get('https://www.card-gorilla.com/search/card?cate=CRD')
time.sleep(2) # 페이지 로딩 대기

main_scroll_cnts = 20 # 총 신용카드 개수 760개, # 수정해야함
sub_scroll_cnts = 3

for cnt in range(main_scroll_cnts):
    more_list = driver.find_element(By.CLASS_NAME, 'lst_more')    # 더보기 클릭
    more_list.click()
    time.sleep(1)

detail_links = driver.find_elements(By.CLASS_NAME, 'b_view')  # 카드 자세히보기 클릭
link_urls = [link.get_attribute('href') for link in detail_links[100:200]]  # 순차적으로 각 카드별 자세히보기 접근. # 수정해야함

card_data = []

for card_index, link in enumerate(link_urls):
    driver.get(link)
    time.sleep(2)

    # 카드 이름, 카드사, 카드 정보
    card_name = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > strong')
    card_brand = driver.find_element(By.CLASS_NAME, 'brand')
    card_detail1 = driver.find_element(By.CLASS_NAME, 'bnf1')
    card_detail2 = driver.find_element(By.CLASS_NAME, 'bnf2')

    print(card_index + 100, card_name.text)   # 수정해야함

    # card number 추출
    current_url = driver.current_url
    card_number = re.search(r'/(\d+)$', current_url)
    if card_number:
        extracted_number = card_number.group(1)
        # print("Extracted number:", extracted_number)

    # 카드 페이지 
    card_page = (card_index + 100 // 10)   # 수정해야함
    
    # card 이미지 ## 최종 수정됨
    img = driver.find_element(By.CSS_SELECTOR, '#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.plate_area > div > img')
    img_url = img.get_attribute('src')


    #스크롤
    body = driver.find_element("tag name", 'body')
    # body.send_keys(Keys.PAGE_DOWN)


    for i in range(sub_scroll_cnts):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.8)

    # 주요혜택 리스트
    describes = driver.find_elements(By.CSS_SELECTOR,
                                    '#q-app > section > div.card_detail.fr-view > section > div > article.cmd_con.benefit > div.lst.bene_area > dl')

    for el in describes:
        el.click()
        time.sleep(0.02)
        for i in range(sub_scroll_cnts):
            body.send_keys(Keys.PAGE_DOWN)
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)

    benefit = []  # 주요 혜택을 담을 리스트
    for el in describes:
        sentences = el.find_elements(By.CSS_SELECTOR, 'dd > div > p, table')
        title = el.find_element(By.CSS_SELECTOR, 'dt > .txt1')
        benefit.append('주요혜택 카테고리: ' + title.text)
        for sentence in sentences:
            if sentence.tag_name == 'p':
                benefit.append(sentence.text)
            elif sentence.tag_name == 'table':
                table_html = sentence.get_attribute('outerHTML')
                benefit.append(table_html)
        # benefit.append('\n')
    

    # 카드 정보 저장
        card_info = {
        'card_num': extracted_number,
        'page': card_page,
        'index': card_index + 100,   # 수정해야함
        'card_img': img_url,
        'name': card_name.text,  
        'company': card_brand.text,
        'detail1': card_detail1.text,  
        'detail2': '연회비: ' + card_detail2.text, 
        'benefit': benefit
    }
    card_data.append(card_info)  # 카드 정보를 리스트에 추가

    # 파일 이름으로 카드 이름 사용
    safe_filename = f'{card_index + 100}.{card_name.text}_{extracted_number}.json'   # 수정해야함
    safe_filename = re.sub(r'[<>:"/\\|?*]', '_', safe_filename)  # 파일 이름에 허용되지 않는 문자 제거
    
    # JSON 파일로 저장
    with open(safe_filename, 'w', encoding='utf-8') as f:
        json.dump(card_info, f, ensure_ascii=False, indent=4)

    print(f"Saved card data to {safe_filename}")
    
    driver.back()
    time.sleep(1)